In [ ]:
import re
import numpy as np
from collections import namedtuple

Gate = namedtuple("Gate", ["in1", "in2", "out", "op"])

op = dict()
op["AND"] = bool.__and__
op["OR"] = bool.__or__
op["XOR"] = bool.__xor__

In [ ]:
test_text = """x00: 1
x01: 0
x02: 1
x03: 1
x04: 0
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1

ntg XOR fgs -> mjb
y02 OR x01 -> tnw
kwq OR kpj -> z05
x00 OR x03 -> fst
tgd XOR rvg -> z01
vdt OR tnw -> bfw
bfw AND frj -> z10
ffh OR nrd -> bqk
y00 AND y03 -> djm
y03 OR y00 -> psh
bqk OR frj -> z08
tnw OR fst -> frj
gnj AND tgd -> z11
bfw XOR mjb -> z00
x03 OR x00 -> vdt
gnj AND wpb -> z02
x04 AND y00 -> kjc
djm OR pbm -> qhw
nrd AND vdt -> hwm
kjc AND fst -> rvg
y04 OR y02 -> fgs
y01 AND x02 -> pbm
ntg OR kjc -> kwq
psh XOR fgs -> tgd
qhw XOR tgd -> z09
pbm OR djm -> kpj
x03 XOR y03 -> ffh
x00 XOR y04 -> ntg
bfw OR bqk -> z06
nrd XOR fgs -> wpb
frj XOR qhw -> z04
bqk OR frj -> z07
y03 OR x01 -> nrd
hwm AND bqk -> z03
tgd XOR rvg -> z12
tnw OR pbm -> gnj"""

In [ ]:
with open("input.txt") as f:
    input_text = f.read()

In [ ]:
def parse_input(text: str):
    initial_values, gates_str = text.strip().split("\n\n")
    wires = dict()
    gates = []
    for gate_str in gates_str.split("\n"):
        command, out = gate_str.split(" -> ")
        in1, op, in2 = command.split()
        gates.append(Gate(in1, in2, out, op))
        for wire in [in1, in2, out]:
            wires[wire] = None
    for initial_str in initial_values.split("\n"):
        wire, val = initial_str.split(": ")
        wires[wire] = bool(int(val))
    return wires, gates
    

In [ ]:
def is_ready(gate: Gate, wires):
    if wires[gate.in1] == None or wires[gate.in2] == None:
        return False
    return True

def do_op(gate: Gate, wires):
    return op[gate.op](wires[gate.in1], wires[gate.in2])

def get_z(wires: dict):
    zs = []
    for k, v in wires.items():
        if "z" in k:
            zs.append((k, v))
    zs.sort(key=lambda x: x[0])
    ret = 0
    for i, z in enumerate(zs):
        ret += z[1] * 1<<i
    return ret, zs

def process(wires, gates):
    gates_cpy = gates.copy()
    while gates_cpy:
        for i, gate in enumerate(gates_cpy):
            if is_ready(gate, wires):
                wires[gate.out] = do_op(gate, wires)
                gates_cpy.pop(i) 
    return get_z(wires)

def swap_output(gates: list, source_key, target_key):
    fix_gates = gates.copy()
    for i, gate in enumerate(fix_gates):
        if gate.out == target_key:
            target_idx = i
            target_gate = gate
        if gate.out == source_key:
            source_idx = i
            source_gate = gate
    target_gate, source_gate = Gate(source_gate.in1, source_gate.in2, target_key, source_gate.op), Gate(target_gate.in1, target_gate.in2, source_key, target_gate.op)
    
    fix_gates[source_idx], fix_gates[target_idx] = source_gate, target_gate
    return fix_gates

### Part one

In [ ]:
wires, gates = parse_input(input_text)
gate: Gate

process(wires, gates)[0]

### Part two

In [ ]:
outputs_to_swap = [("z11", "rpv"), ("ctg", "rpb"), ("dmh", "z31"), ("dvq", "z38")]

In [ ]:
wires, gates = parse_input(input_text)
gate: Gate
initial_wires = wires.copy()
for out1, out2 in outputs_to_swap:
    gates = swap_output(gates, out1, out2)
x_gates = [gate for gate in gates if gate.in1 == "rtg" or gate.in2 == "rtg"]

x_wires_keys = sorted([k for k in wires.keys() if "x" in k])
y_wires_keys = sorted([k for k in wires.keys() if "y" in k])
for i, x in enumerate(x_wires_keys):
    wires_x = dict().fromkeys(wires, None)
    for key in x_wires_keys:
        wires_x[key] = False
    for key in y_wires_keys:
        wires_x[key] = False
        
    expected = 1<<i
    wires_x[x] = True
    val, zs = process(wires_x, gates)
    if val != expected:
        print(x, val, expected)
        
for i, x in enumerate(y_wires_keys):
    wires_y = dict().fromkeys(wires, None)
    for key in x_wires_keys:
        wires_y[key] = False
    for key in y_wires_keys:
        wires_y[key] = False
        
    expected = 1<<i
    wires_y[x] = True
    val, zs = process(wires_y, gates)
    if val != expected:
        print(x, val, expected)
        

In [ ]:
out = []
for pair in outputs_to_swap:
    out += list(pair)
out.sort()
out
",".join(out)

In [ ]:
x11_gates = [gate for gate in gates if gate.in1 == "x38" or gate.in2 == "x38"]
print(x11_gates)
key = "hhv"
searched_gate = [gate for gate in gates if gate.in1 == key or gate.in2 == key or gate.out == key]
print(searched_gate)
source_key = "ctg"
target_key = "rpb"
for i, gate in enumerate(gates):
    if gate.out == target_key:
        target_idx = i
        target_gate = gate
    if gate.out == source_key:
        source_idx = i
        source_gate = gate
target_gate, source_gate = Gate(source_gate.in1, source_gate.in2, target_key, source_gate.op), Gate(target_gate.in1, target_gate.in2, source_key, target_gate.op)
 
gates_fix2 = gates.copy()
gates_fix2[source_idx], gates_fix2[target_idx] = source_gate, target_gate
